<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
# Install folium if you haven't already
# %pip install folium

import folium

# Define the coordinates of the launch sites
launch_sites = {
    'CCAFS SLC 40': [28.561857, -80.577366],
    'KSC LC 39A': [28.573255, -80.646895],
    'VAFB SLC 4E': [34.632093, -120.610829],
    'CCAFS LC 40': [28.562302, -80.577356]
}

# Create a base map
world_map = folium.Map(location=[28.562302, -80.577356], zoom_start=4)

# Add markers for each launch site
for site, coords in launch_sites.items():
    folium.Marker(
        location=coords,
        popup=f"{site}",
        icon=folium.Icon(icon="rocket", prefix='fa')
    ).add_to(world_map)

# Display the map
world_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Create a base map centered around one of the launch sites
world_map = folium.Map(location=[28.562302, -80.577356], zoom_start=4)

# Add markers for each launch site
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(icon="rocket", prefix='fa')
    ).add_to(world_map)

# Display the map
world_map


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Launch sites data
launch_sites_df = pd.DataFrame({
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745]
})

# Add a circle and marker for each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    # Create a circle for each launch site
    circle = folium.Circle(
        coordinate,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))

    # Create a marker for each launch site
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )

    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [11]:
# Task 2: Mark the success/failed launches for each site on the map

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch record
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    launch_site = record['Launch Site']
    success = record['class']
    
    # Determine the color of the marker
    marker_color = 'green' if success == 1 else 'red'
    
    # Create a marker for each launch record
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Site: {launch_site}\nSuccess: {success}"
    )
    
    # Add the marker to the MarkerCluster
    marker.add_to(marker_cluster)

# Display the map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
marker_cluster = MarkerCluster()


In [12]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a smaller circle for each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Create a smaller circle for each launch site
    circle = folium.Circle(
        coordinate,
        radius=500,  # 500 meters radius to make the circle smaller
        color='#d35400',
        fill=True,
        fill_opacity=0.4
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add the circle to the map
    site_map.add_child(circle)

# Add markers for each launch record
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    
    # Create a marker for each launch record
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Site: {record['Launch Site']}\nSuccess: {'Yes' if record['class'] == 1 else 'No'}"
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the site map
site_map.add_child(marker_cluster)

# Display the map
site_map


<class 'KeyError'>: 'marker_color'

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column 'marker_color' in spacex_df
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Display the first few rows to check the new column
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [15]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Create a new column 'marker_color' in spacex_df
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Display the first few rows to check the new column
spacex_df.head()

# Add a smaller circle for each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Create a smaller circle for each launch site
    circle = folium.Circle(
        coordinate,
        radius=50,  # 500 meters radius to make the circle smaller
        color='#d35400',
        fill=True,
        fill_opacity=0.4
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add the circle to the map
    site_map.add_child(circle)

# Add markers for each launch record
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    
    # Create a marker for each launch record
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Site: {record['Launch Site']}\nSuccess: {'Yes' if record['class'] == 1 else 'No'}"
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the site map
site_map.add_child(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [16]:
# TASK 3: Calculate the distances between a launch site to its proximities
import numpy as np
import pandas as pd

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers. Use 3956 for miles.
    return c * r

proximities = pd.DataFrame({
    'Name': ['Proximity A', 'Proximity B', 'Proximity C'],
    'Lat': [28.5721, 28.5634, 28.6110],
    'Long': [-80.6480, -80.5768, -80.6043]
})

launch_sites_df = pd.DataFrame({
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745]
})

distances = []

for i, launch_site in launch_sites_df.iterrows():
    for j, proximity in proximities.iterrows():
        distance = haversine(launch_site['Lat'], launch_site['Long'], proximity['Lat'], proximity['Long'])
        distances.append([launch_site['Launch Site'], proximity['Name'], distance])

distances_df = pd.DataFrame(distances, columns=['Launch Site', 'Proximity', 'Distance (km)'])

# Display the distances
distances_df


,Launch Site,Proximity,Distance (km)
0,CCAFS LC-40,Proximity A,6.984428
1,CCAFS LC-40,Proximity B,0.133622
2,CCAFS LC-40,Proximity C,6.020218
3,CCAFS SLC-40,Proximity A,7.021386
4,CCAFS SLC-40,Proximity B,0.022657
5,CCAFS SLC-40,Proximity C,5.954258
6,KSC LC-39A,Proximity A,0.167744
7,KSC LC-39A,Proximity B,6.932402
8,KSC LC-39A,Proximity C,5.908503
9,VAFB SLC-4E,Proximity A,3818.996633


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [33]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [18]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers. Use 3956 for miles.
    return c * r

# Coordinates of the launch site (example)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates of the closest coastline point (as provided)
coastline_lat = 28.56353
coastline_lon = -80.56798

# Calculate the distance
distance_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance between the launch site and the closest coastline point: {distance_coastline:.2f} km")


Distance between the launch site and the closest coastline point: 0.93 km


In [19]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers. Use 3956 for miles.
    return c * r

# Coordinates of the launch site (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordinates of the closest coastline point (as provided)
coastline_lat = 28.56353
coastline_lon = -80.56798

# Calculate the distance
distance_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance between the launch site and the closest coastline point: {distance_coastline:.2f} km")

# Create a folium Map object centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a smaller circle for each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Create a smaller circle for each launch site
    circle = folium.Circle(
        coordinate,
        radius=50,  # 500 meters radius to make the circle smaller
        color='#d35400',
        fill=True,
        fill_opacity=0.4
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add the circle to the map
    site_map.add_child(circle)

# Add markers for each launch record
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    
    # Create a marker for each launch record
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Site: {record['Launch Site']}\nSuccess: {'Yes' if record['class'] == 1 else 'No'}"
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the site map
site_map.add_child(marker_cluster)

# Add a marker for the closest coastline point with the distance
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)



Distance between the launch site and the closest coastline point: 0.86 km


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [20]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

<class 'NameError'>: name 'lines' is not defined

In [21]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers. Use 3956 for miles.
    return c * r

# Coordinates of the launch site (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordinates of the closest coastline point (as provided)
coastline_lat = 28.56353
coastline_lon = -80.56798

# Calculate the distance
distance_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance between the launch site and the closest coastline point: {distance_coastline:.2f} km")

# Create a folium Map object centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a smaller circle for each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Create a smaller circle for each launch site
    circle = folium.Circle(
        coordinate,
        radius=50,  # 500 meters radius to make the circle smaller
        color='#d35400',
        fill=True,
        fill_opacity=0.4
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add the circle to the map
    site_map.add_child(circle)

# Add markers for each launch record
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    
    # Create a marker for each launch record
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Site: {record['Launch Site']}\nSuccess: {'Yes' if record['class'] == 1 else 'No'}"
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the site map
site_map.add_child(marker_cluster)

# Add a marker for the closest coastline point with the distance
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the distance marker to the site map
site_map.add_child(distance_marker)

# Create a PolyLine between the launch site and the closest coastline point
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)

# Add the PolyLine to the site map
site_map.add_child(lines)

# Display the map
site_map


Distance between the launch site and the closest coastline point: 0.86 km


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [30]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers. Use 3956 for miles.
    return c * r

# Coordinates of the launch site (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordinates of the closest coastline point
coastline_lat = 28.56353
coastline_lon = -80.56798

# Coordinates of Titusville, FL
titusville_lat = 28.6122
titusville_lon = -80.8076

# Coordinates of the FEC Railway near Cape Canaveral, FL
railway_lat = 28.5721
railway_lon = -80.5853

# Coordinates of the nearest point on Max Brewer Causeway (Route 405)
highway_lat = 28.5721
highway_lon = -80.6473

# Calculate distances
distance_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_titusville = haversine(launch_site_lat, launch_site_lon, titusville_lat, titusville_lon)
distance_railway = haversine(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway = haversine(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

print(f"Distance between the launch site and the closest coastline point: {distance_coastline:.2f} km")
print(f"Distance between the launch site and Titusville, FL: {distance_titusville:.2f} km")
print(f"Distance between the launch site and the FEC Railway: {distance_railway:.2f} km")
print(f"Distance between the launch site and the nearest highway: {distance_highway:.2f} km")

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for the launch site (CCAFS SLC-40)
launch_site_marker = folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='CCAFS SLC-40 Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
)
site_map.add_child(launch_site_marker)

# Add a marker for the closest coastline point
coastline_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline',
    icon=folium.Icon(color='cadetblue', icon='info-sign')
)
site_map.add_child(coastline_marker)

# Add a marker for Titusville, FL
titusville_marker = folium.Marker(
    location=[titusville_lat, titusville_lon],
    popup='Titusville, FL',
    icon=folium.Icon(color='green', icon='info-sign')
)
site_map.add_child(titusville_marker)

# Add a marker for the FEC Railway
railway_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    popup='FEC Railway',
    icon=folium.Icon(color='purple', icon='train')
)
site_map.add_child(railway_marker)

# Add a marker for the nearest highway (Max Brewer Causeway)
highway_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    popup='Nearest Highway (Max Brewer Causeway)',
    icon=folium.Icon(color='orange', icon='road')
)
site_map.add_child(highway_marker)

# Add markers displaying the distances
distance_markers = [
    ((launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2, distance_coastline, 'coastline', '#d35400'),
    ((launch_site_lat + titusville_lat) / 2, (launch_site_lon + titusville_lon) / 2, distance_titusville, 'Titusville', '#d35400'),
    ((launch_site_lat + railway_lat) / 2, (launch_site_lon + railway_lon) / 2, distance_railway, 'railway', '#d35400'),
    ((launch_site_lat + highway_lat) / 2, (launch_site_lon + highway_lon) / 2, distance_highway, 'highway', '#d35400')
]

for lat, lon, distance, label, color in distance_markers:
    distance_marker = folium.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:{color};"><b>{distance:.2f} KM</b></div>'
        )
    )
    site_map.add_child(distance_marker)

# Create PolyLines between the launch site and each location
coordinates_list = [
    ([launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]),
    ([launch_site_lat, launch_site_lon], [titusville_lat, titusville_lon]),
    ([launch_site_lat, launch_site_lon], [railway_lat, railway_lon]),
    ([launch_site_lat, launch_site_lon], [highway_lat, highway_lon])
]

for coordinates in coordinates_list:
    lines = folium.PolyLine(locations=coordinates, weight=1)
    site_map.add_child(lines)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Display the map
site_map


Distance between the launch site and the closest coastline point: 0.86 km
Distance between the launch site and Titusville, FL: 23.18 km
Distance between the launch site and the FEC Railway: 1.29 km
Distance between the launch site and the nearest highway: 6.95 km


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
